In [101]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [102]:
city_wx = pd.read_csv("../WeatherPy/city_wx.csv")
city_wx.head()

,City,Cloudiness,Country Code,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Ponta do Sol,20.0,PT,1.607904e+09,82.0,32.67,-17.10,62.60,4.70
1,Bathsheba,20.0,BB,1.607904e+09,83.0,13.22,-59.52,78.80,6.93
2,Dikson,100.0,RU,1.607904e+09,92.0,73.51,80.55,-2.51,14.23
3,Penzance,83.0,GB,1.607904e+09,87.0,50.12,-5.54,51.80,26.40
4,Adrar,90.0,MR,1.607904e+09,27.0,20.50,-10.07,64.13,7.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [120]:
# Store latitude and longitude in locations

locations = city_wx[["Latitude", "Longitude"]]

humidity = city_wx["Humidity"]

# Plot Heatmap

fig = gmaps.figure()

#create heat layer

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=3)

#add layer

fig.add_layer(heat_layer)

#display figure

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [104]:
ideal_wx = city_wx.loc[(city_wx["Humidity"] >= 20) & (city_wx["Humidity"] <= 60) & \
                       (city_wx["Wind Speed"] <= 10) & (city_wx["Max Temp"] >=70) & \
                       (city_wx["Max Temp"] <= 89) & (city_wx["Cloudiness"] <= 5)]

                                                                            

ideal_wx.dropna()





,City,Cloudiness,Country Code,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
48,New Norfolk,0.0,AU,1.607904e+09,38.0,-42.78,147.06,84.00,3.00
54,Kiryat Gat,0.0,IL,1.607904e+09,37.0,31.61,34.76,75.00,8.05
300,Bozoum,1.0,CF,1.607904e+09,30.0,6.32,16.38,73.02,2.35
311,Kawhia,0.0,NZ,1.607904e+09,42.0,-38.07,174.82,77.00,3.00
361,Yola,0.0,NG,1.607904e+09,24.0,9.20,12.48,77.43,2.48
378,Acajutla,0.0,SV,1.607904e+09,45.0,13.59,-89.83,84.20,2.24
391,Bendigo,3.0,AU,1.607904e+09,53.0,-36.77,144.28,86.00,1.99
492,Kolondiéba,0.0,ML,1.607904e+09,23.0,11.09,-6.89,73.80,3.53
549,Lafiagi,0.0,NG,1.607904e+09,51.0,8.87,5.42,78.93,4.45


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [124]:
hotel_df = ideal_wx[["City", "Country Code", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""

<ipython-input-124-55230247b415>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


In [106]:
hotel_df

,City,Country Code,Latitude,Longitude,Hotel Name
48,New Norfolk,AU,-42.78,147.06,
54,Kiryat Gat,IL,31.61,34.76,
300,Bozoum,CF,6.32,16.38,
311,Kawhia,NZ,-38.07,174.82,
361,Yola,NG,9.20,12.48,
378,Acajutla,SV,13.59,-89.83,
391,Bendigo,AU,-36.77,144.28,
492,Kolondiéba,ML,11.09,-6.89,
549,Lafiagi,NG,8.87,5.42,


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = { 
    "type": "hotel",
    "keyword":"hotel",
    "radius": 5000,
    "key": gkey}


# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City"]
    
    params["location"] = f"{lat},{lng}"


    # assemble url and make API request
    print(f"Retrieving Results for a hotel in {index}, {city}.")
    response = requests.get(base_url, params=params).json()
    
    
    
    # extract results
    results = response['results']
    
    try:
        print(f"A hotel within 5000 meters of the coordinates is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")



Retrieving Results for a hotel in 48, New Norfolk.
Missing field/result... skipping.
------------
Retrieving Results for a hotel in 54, Kiryat Gat.
A hotel within 5000 meters of the coordinates is Desert Gat.
------------
Retrieving Results for a hotel in 300, Bozoum.


/Users/geoffreymatis/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.
------------
Retrieving Results for a hotel in 311, Kawhia.
A hotel within 5000 meters of the coordinates is Kawhia Hotel.
------------
Retrieving Results for a hotel in 361, Yola.
A hotel within 5000 meters of the coordinates is AUN Hotel.
------------
Retrieving Results for a hotel in 378, Acajutla.
A hotel within 5000 meters of the coordinates is Villavela Hotel Boutique.
------------
Retrieving Results for a hotel in 391, Bendigo.
A hotel within 5000 meters of the coordinates is All Seasons Resort Hotel.
------------
Retrieving Results for a hotel in 492, Kolondiéba.


In [110]:
hotel_df

,City,Country Code,Latitude,Longitude,Hotel Name
48,New Norfolk,AU,-42.78,147.06,The Woodbridge
54,Kiryat Gat,IL,31.61,34.76,Desert Gat
300,Bozoum,CF,6.32,16.38,
311,Kawhia,NZ,-38.07,174.82,Kawhia Hotel
361,Yola,NG,9.20,12.48,
378,Acajutla,SV,13.59,-89.83,Villavela Hotel Boutique
391,Bendigo,AU,-36.77,144.28,All Seasons Resort Hotel
492,Kolondiéba,ML,11.09,-6.89,Hotel Dakan
549,Lafiagi,NG,8.87,5.42,One to Z Hotel


In [117]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [118]:
#How to plot this on to a US map

marker_locations = hotel_df[["Latitude", "Longitude"]]


fig = gmaps.figure()

#info box is a list corresponding to each layer value 

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))